# Introduction to Python
# Homework #3
- You MUST submit on Courseworks before it closes
- Email submissions are NOT accepted

# Academic Honesty
* The computer science department has strict polices. Check
the department [web page](http://www.cs.columbia.edu/education/honesty) for details. 
- Do not look at anybody else's source code. Do not show anybody
your source, or leave your source where somebody could see it.
You MUST write your own code.
- For this class, feel free to discuss issues with other people, but suggest waiting an hour or two after a discussion, before writing your code.
-  Cases of non original source will be refered to the Judical Committee.



# tips
- 'self' must be the first arg to every method
- use the 'self.' prefix to refer to instance variables or other methods inside a method 

In [593]:
# copy a dictionary

d = {1:2, 3:4}
dd = d.copy()

d is dd, d == dd, dd

(False, True, {1: 2, 3: 4})

In [594]:
# raise ValueError
raise ValueError('messages')

ValueError: messages

# Problem 1 & 2 - Constraints
- suppose we want to convert between C(Celsius) and F(Fahrenheit), using the equation 9*C = 5*(F-32)
- could write functions 'c2f' and 'f2c'

In [595]:
def c2f(c):
    return( (9.*c + 5. * 32.)/5. )

def f2c(f):
    return( 5.*(f  - 32)/9. )

In [596]:
[c2f(0), c2f(100), f2c(32), f2c(212)]

[32.0, 212.0, 0.0, 100.0]

- to write f2c, we solved the equation for C, and made a function out of the other side of the equation
- to write c2f, we solved for F, ...
- there is another way to think about this 
- rearrange the equation into a symmetric form

```
9*C - 5*F = -32*5
```

- you can think of the equation above as a "constraint" between F and C. if you specify one variable, 
the other's value is determined by the equation. in general, if we have

```
c0*x0 + c1*x1 + ... cN*xN = total
```

- cI are fixed coefficients
- specifying any N of the (N+1) x's will determine the remaining x variable
- ```__init__ and __repr__``` methods are done for you

# define the 'setvar' method on the 'Constraint' class
- 1st arg is self
- 2nd arg is a variable name
    - raise a 'ValueError' if given a bad variable name
- 3rd arg is the new value for the variable
- if only one undefined variable is left, 'fire the constraint'
    - solve for the remaining value
    - print which variable you are solving for
    - print all the variables and values
    - now the constraint is 'spent' and should be discarded
- otherwise just record the new variable value
- do all internal computation in floating point
- consider using helper class 'Var'

In [597]:
class Var:
    "Holds info about each constraint var"
    def __init__(self, name, coe):
        self.name = name
        self.coe = float(coe)
        self.value = None
    
    def __repr__(self):
        return f'{self.coe}*{self.name}(={self.value})'
    
    def dot(self):
        return self.value * self.coe if self.value else 0.0

class Constraint:
    def __init__(self, vcdict, total):
        # dict comprehension
        self.vars = {name: Var(name, coe) for name, coe in vcdict.items()}
        self.total = float(total)
    
    def __repr__(self):
        # display the status of the constraint
        # show which vars have values
        x = ' + '.join([f'{repr(v)}' for k,v in self.vars.items()])
        return f'Constraint({self.total}={x})'
    

    # add your setvar method here
    def setvar(self, varname, val): # shamelessly stealing var names from ValError output below!
        count = 0
        
        # raise error if invalid varname
        if not varname in self.vars: # also stealing the error output because it's pretty
            error = f'varname {varname} is not defined in: {",".join(self.vars.keys())}'
            raise ValueError(error)
        else:
            self.vars[varname].value = float(val)
        
        # run domath if contraint is ready
        for name, coe in self.vars.items():
            if self.vars[name].value == None: count += 1
        if count == 1:
            self.__domath()
            # cleanup
            self.vars = {}
            self.total = 0.0
        
    def __domath(self): # let's play with private functions
        # i have a dict of vars with values and a coe, and a total. 
        # one var does not have a value - I need to solve for this one
        # basically, multiply each var by its coe then add those together
        # now divide that total by the coe of the var missing a value.
        run_total = self.total
        
        for name, coe in self.vars.items():
            if self.vars[name].value == None:
                div_by = self.vars[name].coe
                solv_for = name
            else:
                # taking advantage of that dot function
                run_total = run_total - self.vars[name].dot()
        self.vars[solv_for].value = run_total / div_by
        
        print(f'calculating {solv_for}')
        # I probably could eliminate one of these for loops...
        for name, coe in self.vars.items():
            print(f'{name} = {self.vars[name].value}')
        return

# Example - setup C & F constraint

- given equation 9*C - 5*F = -32*5, we can setup a constraint like this:
- string computed by repr method shows C & F initially have no values(=None)

In [598]:
c = Constraint({'C':9, 'F':-5}, -5*32)
c

Constraint(-160.0=9.0*C(=None) + -5.0*F(=None))

In [599]:
# bad variable name - raise an error

c.setvar('foo', 0)
c

ValueError: varname foo is not defined in: C,F

In [600]:
c.setvar('C', 100)

calculating F
C = 100.0
F = 212.0


In [601]:
c = Constraint({'C':9, 'F':-5}, -5*32)
c.setvar('F', 212)

calculating C
C = 100.0
F = 212.0


# more complex example
- 5 constraint variables

In [602]:
d = {f'x{n}':n for n in range(5)}
d

{'x0': 0, 'x1': 1, 'x2': 2, 'x3': 3, 'x4': 4}

In [603]:
c2 = Constraint(d, 1)
c2

Constraint(1.0=0.0*x0(=None) + 1.0*x1(=None) + 2.0*x2(=None) + 3.0*x3(=None) + 4.0*x4(=None))

In [604]:
c2.setvar('x1', 10)
c2

Constraint(1.0=0.0*x0(=None) + 1.0*x1(=10.0) + 2.0*x2(=None) + 3.0*x3(=None) + 4.0*x4(=None))

In [605]:
c2.setvar('x0', 0)
c2

Constraint(1.0=0.0*x0(=0.0) + 1.0*x1(=10.0) + 2.0*x2(=None) + 3.0*x3(=None) + 4.0*x4(=None))

In [606]:
# x2

c2.setvar('x2',20)
c2

Constraint(1.0=0.0*x0(=0.0) + 1.0*x1(=10.0) + 2.0*x2(=20.0) + 3.0*x3(=None) + 4.0*x4(=None))

In [607]:
# only two unset vars left, so setting x3 or x4 
# will fire the constraints

c2.setvar('x4', 30)

calculating x3
x0 = 0.0
x1 = 10.0
x2 = 20.0
x3 = -56.333333333333336
x4 = 30.0


# Problem 3, 4 & 5 - vending machine
- use objects to simulate several vending machines
- money is in units of cents

In [608]:
# stuff needed 

from collections import defaultdict
from enum import Enum
import time

class Coin(Enum):
    quarter = 25
    dime = 10
    nickel = 5
    penny = 1
    
class Item(Enum):
    coke = 55
    pepsi = 74 
    chunky = 88
    chips = 20

# Making change
- the customer always pays with a dollar. all items cost less than a dollar, so the vending machine should return the appropriate change in coins. 
- if the vending machine doesn't have enough coins to make change, the sale can not go thru
- the Changer class will compute change, but it does not quite do what we need. the makeChange method returns two things. first a set of coins which may be the exact amount needed. in this case the 2nd return value, 'owe' will be 0. if there are not enough coins in inventory, the largest amount of coins less than what is owed the customer is returned, and the owe return value will be nonzero. 
- In either case, the inventory of coins held will be decremented. 

In [609]:
class Changer:
    
    def __init__(self, inventory):
        self.inventory = defaultdict(int)
        # put in whatever inventory was specified,
        for k,v in inventory.items():
            self.inventory[k] = v
            
    def __repr__(self):
        # default printer is pretty useless, so make one that is helpful
        # shows the current coin inventory
        lst = [f'{coin.name}={ self.inventory[coin] }' for coin in Coin]
        return 'Changer<' + ','.join(lst) + '>'

    def makeChange(self, price):
        "assume item price is always less than a dollar, and the customer pays with a dollar"
        owe = 100 - price
        ans = defaultdict(int)
        for coin in Coin:
            cnt = owe // coin.value
            # can't return coins we don't have
            cntr = min(cnt, self.inventory[coin])
            ans[coin] = cntr
            self.inventory[coin] -= cntr
            owe -= cntr * coin.value
            if owe == 0:
                break

        #  (coins returned, amount still owed, if any)
        # return a regular dict for readability0
        return (dict(ans), owe)

In [610]:
c1 = Changer({Coin.quarter:2,
              Coin.dime:3,
              Coin.nickel:10,
             Coin.penny:1})

c1

Changer<quarter=2,dime=3,nickel=10,penny=1>

In [611]:
# had enough coins to make 70, so owe = 0 

c1.makeChange(30)

({<Coin.quarter: 25>: 2, <Coin.dime: 10>: 2}, 0)

In [612]:
# inventory is smaller

c1

Changer<quarter=0,dime=1,nickel=10,penny=1>

In [613]:
# don't have 90 in change, so returned 61 in change, and owe = 29

c1.makeChange(10)

({<Coin.quarter: 25>: 0,
  <Coin.dime: 10>: 1,
  <Coin.nickel: 5>: 10,
  <Coin.penny: 1>: 1},
 29)

In [614]:
# inventory smaller

c1

Changer<quarter=0,dime=0,nickel=0,penny=0>

# the problem
- when the vending machine can't return enough change, the sale should be cancelled. but calling Changer.makeChange() modifies the coin inventory in Changer, which is wrong. So Changer has useful functionality, it just isn't quite right for our needs.

# the solution
- you can not modify the Changer class. instead...

# define class Changer2, which inherits from Changer

# define Changer2 method makeChangeCarefully(self, price)
- if there are enough coins, it returns them, like makeChange
- but otherwise, it raises a ValueError, and does NOT change the coin inventory.
- makeChangeCarefully() should call makeChange()

In [615]:
class Changer2(Changer):
    def makeChangeCarefully(self, price):
        # so quick thoughts: I could copy the current inventory dict to a new dict
        # count how much change I can make and compare to what is owed,
        # or I could save the dict, run makeChange and if the return > 0
        # I can restore the dict - which seems really easy
        # import copy
        
        # avoiding shallow copy for now - try it later
        # turns out shallow copy is fine - int = singleton
        inv_copy = self.inventory.copy()
        inv = defaultdict(int)
        
        inv, balance = self.makeChange(price) # wonder if this works
        
        if balance > 0:
            self.inventory = inv_copy
            raise ValueError('Not enough change in stock: ' + str(dict(self.inventory)))
        else:
            return (dict(inv))

In [616]:
c2 = Changer2({Coin.quarter:2,
              Coin.dime:3,
              Coin.nickel:10,
             Coin.penny:1})

c2

Changer<quarter=2,dime=3,nickel=10,penny=1>

In [617]:
# had enough coins to make 70

c2.makeChangeCarefully(30)

{<Coin.quarter: 25>: 2, <Coin.dime: 10>: 2}

In [618]:
# inventory is smaller

c2

Changer<quarter=0,dime=1,nickel=10,penny=1>

In [619]:
# don't have 90 in change, so raise ValueError
# inventory must not be decremented

c2.makeChangeCarefully(10)

ValueError: Not enough change in stock: {<Coin.quarter: 25>: 0, <Coin.dime: 10>: 1, <Coin.nickel: 5>: 10, <Coin.penny: 1>: 1}

In [620]:
# Inventory didn't change

c2

Changer<quarter=0,dime=1,nickel=10,penny=1>

# define class Venditem - represents a type of item for sale
- has two object/instance variables
    - name, quantity
- define four methods
    - `method __init__` loads data into the object/instance variables
        - def `__init__`(self, name, quantity):
    - `method __repr__`(self)
        - controls how venditem prints
        - see examples below
    - method sale(self)
        - decrement the quantity 

In [621]:
class Venditem:
    def __init__(self, name, quantity):
        self.name = name
        self.quantity = quantity
    
    def __repr__(self):
        return f'Venditem(name={self.name}, quantity={self.quantity})'
    
    def sale(self):
        self.quantity -= 1

In [622]:
# __repr__ method shows object status

vi  =  Venditem('coke', 3)
vi2 = Venditem('pepsi', 1)

vi, vi2

(Venditem(name=coke, quantity=3), Venditem(name=pepsi, quantity=1))

In [623]:
# sale method decrements quantity instance variable

vi.sale()
vi

Venditem(name=coke, quantity=2)

# class Vendmachine 
- Vendmachine has two object/instance variables
    - 'cash' - the amount of money the machine has collected from item sales
    - 'items' - a dictionary, where keys are the name of an item, and the values are the Venditem object
- define 'buy' method on Vendmachine
    - buy(self, name)
        - 'name' is 'coke', 'pepsi', etc
        - customer always pays with a dollar
        - there are several cases, depending item and coin inventory. each case should be logged, using provided 'log' method.
        - cases where sale fails
            - customer asks for an item not carried
            - customer asks for an item whose quantity is 0 - out of stock
            - machine can't return sufficient change. Changer2 will throw a ValueError - it must be caught in a try
            - everything ok, sell the item, decrement item quantity and change inventory(Changer2 will handle this). 
        - 'buy' return value
            - if sale completes - the change due 
            - if sail fails - None
        - when a sale goes thru, a Sale object should be created, and appended to the sales list.
        - see examples below 

In [624]:
class Sale:
    "One created for each sale"
    def __init__(self, vmname, itemname):
        self.vmname = vmname
        self.itemname = itemname
        self.time = time.strftime('%X %x %Z')
        
    def __repr__(self):
        return f'{self.itemname} sold by {self.vmname}  for {Item[self.itemname].value}'

In [625]:
# output of __repr__ method

Sale('butler', 'coke')

coke sold by butler  for 55

In [626]:
class Vendmachine:
    
    def __init__(self, name, coins, stock):
        self.name = name
        self.changer = Changer2(coins)
        self.cash = 0
        self.sales = []
        # make a dict
        self.items = dict()
        # this is a very common pattern
        # using some kind of name or id for objects,
        # make a dict with the ids as keys, objects as values
        for i in stock:
            self.items[i.name] = i

    def time(self):
        return time.strftime('%X %x %Z')

    def log(self, msg):
        # this could go to a file
        print(msg)

    def status(self):
        print(f'cash collected: {self.cash}')
        print(self.changer)
        for k in self.items:
            print(self.items[k])
        for s in self.sales:
            print(s)

    # add buy method here
    def buy(self, name):
        # so check if name in self.items
        # log error to catch
        try:
            if name not in self.items:
                raise Exception(f'{self.time()} dont carry {name}')
            if self.items[name].quantity == 0:
                raise Exception(f'{self.time()} {name} out of stock')
                
            chg_due = self.changer.makeChangeCarefully(Item[name].value)
            self.items[name].sale()
            self.sales.append(Sale(self.name, name))
            self.cash = self.cash + Item[name].value
            print(f'{self.time()} {self.sales[-1]}')
            return chg_due
            
        except ValueError:
            self.log(f" {self.time()} can't make change for {name}")
        except Exception as err:
            self.log(err)
            
        return None
            

In [627]:
# make stock for sale and load vendmachine

vi = Venditem('coke', 3)
vi2 = Venditem('pepsi', 1)
vi3 = Venditem('chunky', 2)
vi4 = Venditem('chips', 1)
stock = [vi, vi2, vi3, vi4]

coins = {Coin.quarter:2,
              Coin.dime:3,
              Coin.nickel:10,
             Coin.penny:1}


butler = Vendmachine("butler", coins, stock)
pupin = Vendmachine("pupin", coins, stock)
butler.status()

cash collected: 0
Changer<quarter=2,dime=3,nickel=10,penny=1>
Venditem(name=coke, quantity=3)
Venditem(name=pepsi, quantity=1)
Venditem(name=chunky, quantity=2)
Venditem(name=chips, quantity=1)


In [628]:
butler.buy('mt dew')

17:56:34 02/15/20 EST dont carry mt dew


In [629]:
butler.buy('chunky')

 17:56:34 02/15/20 EST can't make change for chunky


In [630]:
butler.buy('coke')

17:56:36 02/15/20 EST coke sold by butler  for 55


{<Coin.quarter: 25>: 1, <Coin.dime: 10>: 2}

In [631]:
butler.buy('pepsi')

17:56:36 02/15/20 EST pepsi sold by butler  for 74


{<Coin.quarter: 25>: 1,
 <Coin.dime: 10>: 0,
 <Coin.nickel: 5>: 0,
 <Coin.penny: 1>: 1}

In [632]:
butler.buy('pepsi')

17:57:16 02/15/20 EST pepsi out of stock


In [633]:
butler.status()

cash collected: 129
Changer<quarter=0,dime=1,nickel=10,penny=0>
Venditem(name=coke, quantity=2)
Venditem(name=pepsi, quantity=0)
Venditem(name=chunky, quantity=2)
Venditem(name=chips, quantity=1)
coke sold by butler  for 55
pepsi sold by butler  for 74


In [634]:
butler.buy('pepsi')

17:57:18 02/15/20 EST pepsi out of stock


In [635]:
butler.buy('chunky')

 17:57:19 02/15/20 EST can't make change for chunky


In [636]:
butler.status()

cash collected: 129
Changer<quarter=0,dime=1,nickel=10,penny=0>
Venditem(name=coke, quantity=2)
Venditem(name=pepsi, quantity=0)
Venditem(name=chunky, quantity=2)
Venditem(name=chips, quantity=1)
coke sold by butler  for 55
pepsi sold by butler  for 74


In [637]:
pupin.buy('coke')

17:57:22 02/15/20 EST coke sold by pupin  for 55


{<Coin.quarter: 25>: 1, <Coin.dime: 10>: 2}

In [638]:
pupin.buy('chips')

17:57:23 02/15/20 EST chips sold by pupin  for 20


{<Coin.quarter: 25>: 1, <Coin.dime: 10>: 1, <Coin.nickel: 5>: 9}

In [639]:
pupin.status()

cash collected: 75
Changer<quarter=0,dime=0,nickel=1,penny=1>
Venditem(name=coke, quantity=1)
Venditem(name=pepsi, quantity=0)
Venditem(name=chunky, quantity=2)
Venditem(name=chips, quantity=0)
coke sold by pupin  for 55
chips sold by pupin  for 20


# define class Aggregate
- takes a list of vending machines
- report() method prints a sales summary

In [640]:
class Aggregate():
    def __init__(self, machines):
        # in total I can make a set of machines - needed?
        # a dict of items sold and how many of each
        # a total of cash
        
        # from report can get items sold and quantity to output
        # re-use Item to list the value
        self.machines = dict()
        self.sales = []
        self.total_cash = 0
        self.aggr_sales = defaultdict(int)
        
        # iterator from Vendmachines - this is handy
        for i in machines:
            self.machines[i.name] = i
        
        # this is the ugliest thing ever
        # builds set of machines, sales objects, aggr sales and cash
        for key, value in self.machines.items():
            self.sales.append(value.sales)
            self.total_cash = self.total_cash + value.cash
            for i in value.sales:
                self.aggr_sales[i.itemname] +=1
    
    def report(self):
        tally = dict(self.aggr_sales)
        
        for key, value in tally.items():
            print(f'{value} sales of {key} at {Item[key].value}')
        print(f'total cash collected: {self.total_cash}')
              
        return
    
        

In [641]:
a = Aggregate([butler, pupin])

In [642]:
a.report()

2 sales of coke at 55
1 sales of pepsi at 74
1 sales of chips at 20
total cash collected: 204


# Just for fun...


# sketchpad(1963)
- IMHO, [sketchpad](https://en.wikipedia.org/wiki/Sketchpad) is the greatest CS PhD thesis ever. 
- among other things, [Ivan Sutherland](https://en.wikipedia.org/wiki/Ivan_Sutherland) invented constraint systems, interactive computer graphics, CAD, object oriented programming, and visual programming
- the computer he used had 33K of memory!
- if you have a few minutes sometime, watch this short [sketchpad video from summer 1962](https://www.youtube.com/watch?v=495nCzxM9PI). the speaker is [Alan Kay](https://en.wikipedia.org/wiki/Alan_Kay), of Xerox PARC fame

# analyze function huh
- This is NOT graded - don't spend more than 5 minutes on it
- huh's arg takes ints greater than 1
- what does huh compute?
- how does the computation work?
- is huh pythonic?
- math module [doc](https://docs.python.org/3/library/math.html)

In [52]:
import math

math.fabs(4.3), math.fabs(-3.4)

(4.3, 3.4)

In [53]:
math.ceil(2.3), math.ceil(0.0)

(3, 0)

In [54]:
def huh(n):
    answer = True
    count = 2
    while count < n:
        answer = [False,True][math.ceil
                              (math.fabs(math.sin(n%count)))]
        count = [count + 1, n][~answer]
    return answer